# Low Rank Adapaptation (LoRa) of a FLUX model

## Imports + Settings

In [1]:
# Install compatible versions - balance between too new and too old
%pip install pillow pillow-avif-plugin openai "diffusers==0.30.2" "huggingface_hub==0.24.6" accelerate peft datasets dotenv rich tqdm ipywidgets

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Configure logging

from rich.console import Console
from rich.logging import RichHandler
import logging

console = Console()

logging.basicConfig(
    level="INFO",
    format="%(message)s",
    datefmt="[%X]",
    handlers=[RichHandler(console=console)]
)
log = logging.getLogger("rich")

In [3]:
from pathlib import Path
import json, os, re, shutil

IMAGES_OLD_DIR = Path("./images-old")
IMAGES_NEW_DIR = Path("./images-new")

WORK_DIR = Path("./work")
CONVERTED_DIR = WORK_DIR / "converted"
CONVERTED_OLD = CONVERTED_DIR / "old"
CONVERTED_NEW = CONVERTED_DIR / "new"
LABELS_DIR = WORK_DIR / "labels"
DATASET_DIR = WORK_DIR / "dataset"
LORA_OUTPUT_DIR = WORK_DIR / "lora-weights"

for p in [WORK_DIR, CONVERTED_OLD, CONVERTED_NEW, LABELS_DIR, DATASET_DIR, LORA_OUTPUT_DIR]:
    p.mkdir(parents=True, exist_ok=True)

TARGET_EXT = ".jpg"

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "YOUR_OPENAI_API_KEY_HERE")
OPENAI_MODEL = "gpt-4o-mini"

LABEL_SCHEMA = [
    "perspective (e.g., top-down, eye-level, low-angle)",
    "background (describe setting, simplicity vs clutter)",
    "lighting (e.g., soft, harsh, high-key, low-key, golden-hour)",
    "color palette (e.g., muted, vibrant, pastel, monochrome)",
    "texture treatment (e.g., painterly, grainy, smooth)",
    "composition (e.g., centered subject, rule of thirds, negative space)",
    "mood (e.g., nostalgic, cheerful, moody, futuristic)",
]

BASE_MODEL_ID = "runwayml/stable-diffusion-v1-5"
USE_8BIT_ADAM = True
MIXED_PRECISION = "fp16"
SEED = 42

log.info("Settings loaded.")




[08:31:01] INFO     Settings loaded.                                                               ]8;id=699387;file:///tmp/ipykernel_16049/3235581259.py\3235581259.py]8;;\:]8;id=639408;file:///tmp/ipykernel_16049/3235581259.py#38\38]8;;\

## Convert images to JPGs

In [4]:
from PIL import Image
from tqdm.notebook import tqdm

def convert_folder(src: Path, dst: Path, target_ext: str = ".jpg"):
    count = 0
    for p in tqdm(sorted(src.rglob("*"))):
        if p.is_dir() or p.name.startswith("."):
            continue
        if p.suffix.lower() not in [".jpg", ".jpeg", ".png", ".avif", ".webp"]:
            # skip non-image files explicitly
            continue
        try:
            with Image.open(p) as im:
                im = im.convert("RGB")  # unify colorspace
                out_path = dst / (p.stem + target_ext)
                out_path.parent.mkdir(parents=True, exist_ok=True)
                if target_ext.lower() == ".jpg":                    
                    im.save(out_path, format="JPEG", quality=95, optimize=True)
                else:
                    im.save(out_path)
                count += 1
        except Exception as e:
            print(f"⚠️ Failed to convert {p}: {e}")
    return count

n_old = convert_folder(IMAGES_OLD_DIR, CONVERTED_OLD, TARGET_EXT)
n_new = convert_folder(IMAGES_NEW_DIR, CONVERTED_NEW, TARGET_EXT)
log.info(f"Converted {n_old} old images and {n_new} new images to {TARGET_EXT}.")



  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

[08:31:03] INFO     Converted 16 old images and 31 new images to .jpg.                             ]8;id=853053;file:///tmp/ipykernel_16049/1464447316.py\1464447316.py]8;;\:]8;id=615077;file:///tmp/ipykernel_16049/1464447316.py#28\28]8;;\

# Image labelling

In [5]:
from openai import AzureOpenAI
import os
import time
import base64

from dotenv import load_dotenv
load_dotenv()

AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_DEPLOYMENT_NAME = os.getenv("AZURE_DEPLOYMENT_NAME")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

client = AzureOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
)

LABEL_SCHEMA = [
    "food item (main dish, ingredient, or drink that appears)",
    "perspective (e.g., top-down, eye-level, low-angle)",
    "background (describe setting, simplicity vs clutter)",
    "lighting (e.g., soft, harsh, high-key, low-key, golden-hour)",
    "color palette (e.g., muted, vibrant, pastel, monochrome)",
    "texture treatment (e.g., painterly, grainy, smooth)",
    "composition (e.g., centered subject, rule of thirds, negative space)",
    "mood (e.g., nostalgic, cheerful, moody, futuristic)",
]


def b64_image(image_path: Path) -> str:
    with open(image_path, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")

def load_existing_labels(jsonl_path: Path) -> set:
    """Load already labeled filenames from existing JSONL file"""
    labeled_files = set()
    if jsonl_path.exists():
        with open(jsonl_path, "r", encoding="utf-8") as f:
            for line in f:
                try:
                    data = json.loads(line.strip())
                    if "filename" in data:
                        labeled_files.add(str(Path(data["filename"]).resolve()))
                except json.JSONDecodeError:
                    continue
    return labeled_files

def label_image(img_path: Path, schema):
    img_b64 = b64_image(img_path)
    schema_bullets = "\n".join([f"- {s}" for s in schema])
    system_prompt = (
        "You are a meticulous food and style image annotator. "
        "Return a strict JSON object with fields exactly matching the requested schema."
    )
    user_prompt = f"""
Analyze the image and label it for the following attributes:
{schema_bullets}

Rules:
- Return strictly valid JSON (no markdown).
- For 'food item', use the most specific name possible (e.g., 'spaghetti carbonara', 'latte art', 'sushi roll').
- For style attributes, use short descriptive phrases.
- Include a 'notes' field for anything noteworthy.
"""
    try:
        resp = client.chat.completions.create(
            model=AZURE_DEPLOYMENT_NAME,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": [
                    {"type": "text", "text": user_prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{img_b64}"}}
                ]},
            ],
            temperature=0.2,
        )
        text = resp.choices[0].message.content
        try:
            return json.loads(text)
        except Exception:
            m = re.search(r"\{[\s\S]*\}", text)
            log.error(f"⚠️ Non-JSON response for {img_path}:\n{text}")
            return json.loads(m.group(0)) if m else {"raw": text}
    except Exception as e:
        return {"error": str(e)}

def label_folder(src: Path, out_jsonl: Path, max_images: int = 10, override: bool = False):
    n = 0
    skipped = 0
    
    # Load existing labels if not overriding
    labeled_files = set() if override else load_existing_labels(out_jsonl)
    
    # Open file in append mode if not overriding and file exists
    mode = "w" if override or not out_jsonl.exists() else "a"
    
    with open(out_jsonl, mode, encoding="utf-8") as out:
        for p in tqdm(sorted(src.glob(f"*{TARGET_EXT}"))):
            if n >= max_images:
                print(f"⏹️ Stopping after {max_images} images")
                break
                
            # Check if already labeled
            if str(p.resolve()) in labeled_files:
                log.info(f"⏭️ Skipping {p.name} (already labeled)")
                skipped += 1
                continue
                
            log.info(f"Labeling {p}...")
            result = label_image(p, LABEL_SCHEMA)
            rec = {"filename": str(p.resolve()), "labels": result}
            out.write(json.dumps(rec, ensure_ascii=False) + "\n")
            n += 1
            time.sleep(0.2)            
    
    print(f"✅ Labeled {n} new images, skipped {skipped} already labeled → {out_jsonl}")

print("Labeling images via Azure OpenAI...")
label_folder(CONVERTED_NEW, LABELS_DIR / "new_labels.jsonl", max_images=50, override=False)
label_folder(CONVERTED_OLD, LABELS_DIR / "old_labels.jsonl", max_images=50, override=False)
print("🎉 Done.")

Labeling images via Azure OpenAI...


  0%|          | 0/31 [00:00<?, ?it/s]

           INFO     ⏭️ Skipping ananassalsa-met-kiwi-jalapeno-en-koriander.jpg (already labeled)    ]8;id=405477;file:///tmp/ipykernel_16049/412114492.py\412114492.py]8;;\:]8;id=810207;file:///tmp/ipykernel_16049/412114492.py#107\107]8;;\

           INFO     ⏭️ Skipping andijviestamp-met-gerookte-kip-en-zongedroogde-tomaat.jpg (already  ]8;id=634248;file:///tmp/ipykernel_16049/412114492.py\412114492.py]8;;\:]8;id=129706;file:///tmp/ipykernel_16049/412114492.py#107\107]8;;\
                    labeled)                                                                                       

           INFO     ⏭️ Skipping bananenbrood-choco-kokostopping-van-giorgias-kitchen.jpg (already   ]8;id=326438;file:///tmp/ipykernel_16049/412114492.py\412114492.py]8;;\:]8;id=533091;file:///tmp/ipykernel_16049/412114492.py#107\107]8;;\
                    labeled)                                                                                       

...

           INFO     ⏭️ Skipping pineapple-slush.jpg (already labeled)                               ]8;id=859941;file:///tmp/ipykernel_16049/412114492.py\412114492.py]8;;\:]8;id=484955;file:///tmp/ipykernel_16049/412114492.py#107\107]8;;\

           INFO     ⏭️ Skipping pitabroodjes-met-lamsvlees-en-tomatenrelish.jpg (already labeled)   ]8;id=268874;file:///tmp/ipykernel_16049/412114492.py\412114492.py]8;;\:]8;id=45732;file:///tmp/ipykernel_16049/412114492.py#107\107]8;;\

           INFO     ⏭️ Skipping pittige-vispakketjes.jpg (already labeled)                          ]8;id=579200;file:///tmp/ipykernel_16049/412114492.py\412114492.py]8;;\:]8;id=675577;file:///tmp/ipykernel_16049/412114492.py#107\107]8;;\

           INFO     ⏭️ Skipping roergebakken-spruitjes.jpg (already labeled)                        ]8;id=362495;file:///tmp/ipykernel_16049/412114492.py\412114492.py]8;;\:]8;id=455439;file:///tmp/ipykernel_16049/412114492.py#107\107]8;;\

✅ Labeled 0 new images, skipped 16 already labeled → work/labels/old_labels.jsonl
🎉 Done.


# Image captions

In [6]:
def captions_from_labels(jsonl_path: Path, out_dir: Path) -> int:
    """
    Reads a JSONL file with Azure OpenAI labeling results and builds
    caption .txt files next to each image for LoRA training.

    Captions include the food item first, followed by style attributes.
    Uses tqdm for progress and rich logging for clean notebook output.
    """
    out_dir.mkdir(parents=True, exist_ok=True)
    dst_img_dir = out_dir / "new"
    dst_img_dir.mkdir(parents=True, exist_ok=True)

    count = 0

    # Count lines first for tqdm total
    total = sum(1 for _ in open(jsonl_path, "r", encoding="utf-8"))

    log.info(f"📑 Building captions from {jsonl_path} ({total} records)...")

    with open(jsonl_path, "r", encoding="utf-8") as f:
        for line in tqdm(f, total=total, desc="Generating captions", leave=True):
            rec = json.loads(line)
            fp = Path(rec["filename"])
            labels = rec.get("labels", {})

            food = labels.get("food item") or labels.get("food item (main dish, ingredient, or drink that appears)")
            perspective = labels.get("perspective")
            background = labels.get("background")
            lighting = labels.get("lighting")
            palette = labels.get("color palette")
            texture = labels.get("texture treatment")
            composition = labels.get("composition")
            mood = labels.get("mood")

            parts = []
            if food:
                parts.append(f"a photo of {food}")
            if composition:
                parts.append(f"composition: {composition}")
            if perspective:
                parts.append(f"perspective: {perspective}")
            if lighting:
                parts.append(f"lighting: {lighting}")
            if palette:
                parts.append(f"color palette: {palette}")
            if texture:
                parts.append(f"texture: {texture}")
            if background:
                parts.append(f"background: {background}")
            if mood:
                parts.append(f"mood: {mood}")

            caption = ", ".join(parts) + "."
            if not caption.strip() or caption == ".":
                caption = "A food scene with distinctive style."

            out_img_path = dst_img_dir / fp.name
            out_txt_path = dst_img_dir / (fp.stem + ".txt")

            try:
                shutil.copy2(fp, out_img_path)
                with open(out_txt_path, "w", encoding="utf-8") as out:
                    out.write(caption)
                count += 1
            except Exception as e:
                log.error(f"⚠️ Failed to process {fp.name}: {e}")

    log.info(f"✅ Wrote captions for {count} images into {dst_img_dir}")
    return count

n1 = captions_from_labels(LABELS_DIR / "new_labels.jsonl", DATASET_DIR)
n2 = captions_from_labels(LABELS_DIR / "old_labels.jsonl", DATASET_DIR)
log.info(f"🎉 Done. Total captions created: {n1 + n2}")

           INFO     📑 Building captions from work/labels/new_labels.jsonl (31 records)...         ]8;id=788204;file:///tmp/ipykernel_16049/1215574807.py\1215574807.py]8;;\:]8;id=158619;file:///tmp/ipykernel_16049/1215574807.py#18\18]8;;\

Generating captions:   0%|          | 0/31 [00:00<?, ?it/s]

[08:31:05] INFO     ✅ Wrote captions for 31 images into work/dataset/new                          ]8;id=488808;file:///tmp/ipykernel_16049/1215574807.py\1215574807.py]8;;\:]8;id=136809;file:///tmp/ipykernel_16049/1215574807.py#68\68]8;;\

           INFO     📑 Building captions from work/labels/old_labels.jsonl (16 records)...         ]8;id=788363;file:///tmp/ipykernel_16049/1215574807.py\1215574807.py]8;;\:]8;id=415905;file:///tmp/ipykernel_16049/1215574807.py#18\18]8;;\

Generating captions:   0%|          | 0/16 [00:00<?, ?it/s]

           INFO     ✅ Wrote captions for 16 images into work/dataset/new                          ]8;id=911460;file:///tmp/ipykernel_16049/1215574807.py\1215574807.py]8;;\:]8;id=20090;file:///tmp/ipykernel_16049/1215574807.py#68\68]8;;\

           INFO     🎉 Done. Total captions created: 47                                            ]8;id=893590;file:///tmp/ipykernel_16049/1215574807.py\1215574807.py]8;;\:]8;id=747935;file:///tmp/ipykernel_16049/1215574807.py#73\73]8;;\

# Model training

In [7]:
# Check PyTorch version and fix if needed
import torch
print(f"Current PyTorch version: {torch.__version__}")

# Check if we have a proper PyTorch installation
try:
    # Test basic PyTorch functionality
    x = torch.tensor([1.0, 2.0])
    print(f"PyTorch tensor test: {x}")
    print(f"CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"CUDA device count: {torch.cuda.device_count()}")
        print(f"Current CUDA device: {torch.cuda.current_device()}")
except Exception as e:
    print(f"PyTorch test failed: {e}")

# The issue might be that the development version isn't recognized by transformers
# Let's patch the version check by setting a compatible version string
import sys
if hasattr(torch, '__version__'):
    original_version = torch.__version__
    if 'a0' in torch.__version__:
        print(f"Detected development PyTorch version: {original_version}")
        print("Patching version string for compatibility...")
        # Set a stable version string that transformers will accept
        torch.__version__ = "2.1.0"
        print(f"Patched PyTorch version to: {torch.__version__}")

Current PyTorch version: 2.1.0a0+b5021ba
PyTorch tensor test: tensor([1., 2.])
CUDA available: True
CUDA device count: 1
Current CUDA device: 0
Detected development PyTorch version: 2.1.0a0+b5021ba
Patching version string for compatibility...
Patched PyTorch version to: 2.1.0


In [ ]:
import torch, random
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from transformers import CLIPTextModel, CLIPTokenizer
from diffusers import AutoencoderKL, UNet2DConditionModel, DDPMScheduler
from accelerate import Accelerator
from peft import LoraConfig, get_peft_model

random.seed(SEED)
torch.manual_seed(SEED)

TRAIN_BATCH_SIZE = 2
LR = 1e-4
MAX_STEPS = 1000
IMG_RES = 512
LORA_R = 8
LORA_ALPHA = 16
LORA_DROPOUT = 0.05

log.info("Loading models...")
tokenizer = CLIPTokenizer.from_pretrained(BASE_MODEL_ID, subfolder="tokenizer")
text_encoder = CLIPTextModel.from_pretrained(BASE_MODEL_ID, subfolder="text_encoder")
vae = AutoencoderKL.from_pretrained(BASE_MODEL_ID, subfolder="vae")
unet = UNet2DConditionModel.from_pretrained(BASE_MODEL_ID, subfolder="unet")

# LoRA configuration for style transfer
# Using UNET_TUNING because we only want to fine-tune the UNet attention layers
# while keeping text encoder and VAE frozen to preserve base language understanding
lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=["to_q", "to_k", "to_v", "to_out.0"],  # UNet attention modules
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="UNET_TUNING",  # Correct for style LoRA - trains UNet only
)
unet = get_peft_model(unet, lora_config)
log.info("LoRA applied to UNet")

class CaptionImageDataset(Dataset):
    def __init__(self, img_dir: Path, tokenizer, size=512):
        self.paths = sorted((img_dir).glob("*.jpg"))
        self.tokenizer = tokenizer
        self.size = size
        log.info(f"Dataset created with {len(self.paths)} images")
    def __len__(self): return len(self.paths)
    def __getitem__(self, idx):
        img_path = self.paths[idx]
        txt_path = img_path.with_suffix(".txt")
        with open(txt_path, "r", encoding="utf-8") as f:
            caption = f.read().strip()
        image = Image.open(img_path).convert("RGB").resize((self.size, self.size), Image.BICUBIC)
        import numpy as np
        arr = np.array(image).astype("float32")/255.0
        arr = arr.transpose(2,0,1)
        arr = 2.0 * arr - 1.0
        image_tensor = torch.from_numpy(arr)
        ids = self.tokenizer(caption, truncation=True, padding="max_length",
            max_length=self.tokenizer.model_max_length, return_tensors="pt").input_ids[0]
        return {"pixel_values": image_tensor, "input_ids": ids}

train_ds = CaptionImageDataset(DATASET_DIR / "new", tokenizer, IMG_RES)
train_loader = DataLoader(train_ds, batch_size=TRAIN_BATCH_SIZE, shuffle=True, drop_last=True)

noise_scheduler = DDPMScheduler.from_pretrained(BASE_MODEL_ID, subfolder="scheduler")
accelerator = Accelerator(mixed_precision=MIXED_PRECISION if MIXED_PRECISION in ("fp16","bf16") else "no")
device = accelerator.device
log.info(f"Using device: {device}")

vae.to(device); text_encoder.to(device); unet.to(device)

try:
    if USE_8BIT_ADAM:
        from bitsandbytes.optim import AdamW8bit
        optimizer = AdamW8bit((p for p in unet.parameters() if p.requires_grad), lr=LR)
        log.info("Using 8-bit AdamW optimizer")
    else:
        raise Exception("use torch adamw")
except Exception:
    from torch.optim import AdamW
    optimizer = AdamW((p for p in unet.parameters() if p.requires_grad), lr=LR)
    log.info("Using standard AdamW optimizer")

unet, optimizer, train_loader = accelerator.prepare(unet, optimizer, train_loader)
global_step = 0
unet.train()

log.info("Starting training...")
for epoch in range(999999):
    for batch in train_loader:
        if global_step >= MAX_STEPS:
            break
        with accelerator.accumulate(unet):
            pixel_values = batch["pixel_values"].to(device)
            input_ids = batch["input_ids"].to(device)
            latents = vae.encode(pixel_values).latent_dist.sample() * 0.18215
            noise = torch.randn_like(latents)
            timesteps = torch.randint(0, noise_scheduler.config.num_train_timesteps, (latents.shape[0],), device=device).long()
            noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)
            encoder_hidden_states = text_encoder(input_ids)[0]
            model_pred = unet(noisy_latents, timesteps, encoder_hidden_states).sample
            loss = torch.nn.functional.mse_loss(model_pred, noise, reduction="mean")
            accelerator.backward(loss)
            optimizer.step(); optimizer.zero_grad()
        if accelerator.is_main_process and global_step % 50 == 0:
            log.info(f"step {global_step} | loss {loss.item():.4f}")
        global_step += 1
    if global_step >= MAX_STEPS: break

if accelerator.is_main_process:
    unet.save_pretrained(str(LORA_OUTPUT_DIR))
    log.info(f"Saved LoRA weights to {LORA_OUTPUT_DIR}")

/usr/local/lib/python3.10/dist-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https:/

[08:31:11] INFO     Loading models...                                                              ]8;id=234053;file:///tmp/ipykernel_16049/3526091652.py\3526091652.py]8;;\:]8;id=146316;file:///tmp/ipykernel_16049/3526091652.py#20\20]8;;\

tokenizer_config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

scheduler_config.json:   0%|          | 0.00/308 [00:00<?, ?B/s]

[08:33:01] INFO     Using device: cuda                                                             ]8;id=750800;file:///tmp/ipykernel_16049/3526091652.py\3526091652.py]8;;\:]8;id=681453;file:///tmp/ipykernel_16049/3526091652.py#68\68]8;;\

[08:33:02] INFO     Using standard AdamW optimizer                                                 ]8;id=617889;file:///tmp/ipykernel_16049/3526091652.py\3526091652.py]8;;\:]8;id=291704;file:///tmp/ipykernel_16049/3526091652.py#82\82]8;;\

           INFO     Starting training...                                                           ]8;id=167414;file:///tmp/ipykernel_16049/3526091652.py\3526091652.py]8;;\:]8;id=732052;file:///tmp/ipykernel_16049/3526091652.py#88\88]8;;\

[08:34:31] INFO     step 0 | loss 0.6095                                                          ]8;id=800581;file:///tmp/ipykernel_16049/3526091652.py\3526091652.py]8;;\:]8;id=352944;file:///tmp/ipykernel_16049/3526091652.py#106\106]8;;\